# Generative networks

Recurrent Neural Networks (RNNs) နှင့် Long Short Term Memory Cells (LSTMs) နှင့် Gated Recurrent Units (GRUs) ကဲ့သို့သော gated cell မျိုးကွဲများသည် ဘာသာစကားမော်ဒယ်တစ်ခုအတွက် စနစ်တစ်ခုကို ပံ့ပိုးပေးခဲ့သည်။ အဆိုပါစနစ်သည် စကားလုံးများ၏ အစီအစဉ်ကို သင်ယူနိုင်ပြီး အစီအစဉ်အတွင်း နောက်တစ်ခုထွက်မည့် စကားလုံးကို ခန့်မှန်းပေးနိုင်သည်။ ၎င်းသည် RNNs ကို **ထုတ်လုပ်မှုဆိုင်ရာ လုပ်ငန်းများ** (ဥပမာ - ပုံမှန်စာသားထုတ်လုပ်မှု၊ စက်ဘာသာပြန်ခြင်း၊ ပုံဖော်ပြချက်ရေးခြင်း) အတွက် အသုံးပြုနိုင်စေသည်။

ယခင်ယူနစ်တွင် ဆွေးနွေးခဲ့သည့် RNN architecture တွင် RNN unit တစ်ခုစီသည် နောက်ထွက်မည့် hidden state ကို output အဖြစ် ထုတ်ပေးသည်။ သို့သော်လည်း, RNN unit တစ်ခုစီတွင် output တစ်ခုထပ်မံထည့်သွင်းနိုင်ပြီး, ၎င်းသည် **sequence** (မူရင်း sequence နှင့် အရှည်တူ) ကို output အဖြစ် ထုတ်ပေးနိုင်သည်။ ထို့အပြင်, RNN units များကို အဆင့်တစ်ခုစီတွင် input မလိုအပ်ဘဲ, စတင်အခြေအနေ vector တစ်ခုသာ လက်ခံပြီး, ထို့နောက် output များ၏ sequence တစ်ခုကို ထုတ်ပေးစေနိုင်သည်။

ဒီ notebook တွင်, စာသားထုတ်လုပ်ရန် ကူညီပေးသည့် ရိုးရှင်းသော generative models များကို အဓိကထား၍ လေ့လာသွားမည်ဖြစ်သည်။ ရိုးရှင်းစွာပြောရမည်ဆိုပါက, **character-level network** တစ်ခုကို တည်ဆောက်ကြမည်ဖြစ်ပြီး, ၎င်းသည် စာလုံးတစ်လုံးချင်းစီဖြင့် စာသားကို ထုတ်လုပ်ပေးမည်ဖြစ်သည်။ သင်ကြားမှုအတွင်း, စာသား corpus တစ်ခုကို ယူပြီး, ၎င်းကို စာလုံးအစီအစဉ်များအဖြစ် ခွဲထုတ်ရန် လိုအပ်မည်။


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## အက္ခရာအဆင့် ဝေါဟာရ တည်ဆောက်ခြင်း

အက္ခရာအဆင့် ဖန်တီးမှုကွန်ရက်တစ်ခု တည်ဆောက်ရန်အတွက် စာသားကို စကားလုံးများမဟုတ်ဘဲ တစ်ခုချင်းစီ အက္ခရာများအဖြစ် ခွဲထုတ်ရမည်ဖြစ်သည်။ ယခင်က အသုံးပြုခဲ့သော `TextVectorization` layer သည် အဲဒီလိုလုပ်ဆောင်နိုင်မည်မဟုတ်သောကြောင့် အောက်ပါရွေးချယ်စရာနှစ်ခုရှိသည်-

* [ဒီ Keras ရဲ့ တရားဝင် ဥပမာ](https://keras.io/examples/generative/lstm_character_level_text_generation/) တွင်ဖော်ပြထားသည့်အတိုင်း စာသားကို ကိုယ်တိုင်တင်သွင်းပြီး 'ကိုယ်တိုင်' tokenization ပြုလုပ်ရန်
* `Tokenizer` class ကို အသုံးပြု၍ အက္ခရာအဆင့် tokenization ပြုလုပ်ရန်။

ကျွန်ုပ်တို့သည် ဒုတိယရွေးချယ်စရာကို အသုံးပြုမည်ဖြစ်သည်။ `Tokenizer` သည် စကားလုံးများအဖြစ်လည်း tokenize ပြုလုပ်နိုင်သည့်အတွက် အက္ခရာအဆင့်မှ စကားလုံးအဆင့် tokenization သို့ အလွယ်တကူ ပြောင်းလဲနိုင်သည်။

အက္ခရာအဆင့် tokenization ပြုလုပ်ရန် `char_level=True` parameter ကို ပေးသွင်းရမည်-


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

ကျွန်ုပ်တို့ **အဆင့်သတ်မှတ်ခြင်း၏ အဆုံး** ကိုဖော်ပြရန် `<eos>` ဟုခေါ်သော အထူးအမှတ်အသားတစ်ခုကိုလည်း အသုံးပြုလိုပါသည်။ ဒါကို အဘိဓာန်ထဲသို့ ကိုယ်တိုင်ထည့်သွင်းလိုက်ရအောင်:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## RNN ကိုသင်ကြားပြီး ခေါင်းစဉ်များကို ဖန်တီးရန်

RNN ကို သတင်းခေါင်းစဉ်များ ဖန်တီးရန် သင်ကြားပုံမှာ အောက်ပါအတိုင်း ဖြစ်ပါတယ်။ တစ်ခုချင်းစီအဆင့်မှာ RNN ထဲသို့ ထည့်မည့် ခေါင်းစဉ်တစ်ခုကို ရွေးပြီး၊ အင်ပွတ်အက္ခရာတစ်ခုစီအတွက် နောက်ထွက်အက္ခရာကို ဖန်တီးရန် ကွန်ယက်ကို မေးမြန်းပါမည်။

![RNN က 'HELLO' စကားလုံးကို ဖန်တီးနေသည်ကို ပြသသော ပုံ။](../../../../../translated_images/my/rnn-generate.56c54afb52f9781d.webp)

အကြောင်းအရာ၏ နောက်ဆုံးအက္ခရာအတွက် `<eos>` token ကို ဖန်တီးရန် ကွန်ယက်ကို မေးမြန်းပါမည်။

ဒီနေရာမှာ အသုံးပြုမည့် generative RNN နှင့် အခြား RNN များ၏ အဓိကကွာခြားချက်မှာ RNN ၏ နောက်ဆုံး cell မှသာမက၊ အဆင့်တစ်ခုချင်းစီမှ ထွက်လာသော output ကိုလည်း အသုံးပြုမည်ဖြစ်သည်။ ဒါကို RNN cell တွင် `return_sequences` parameter ကို သတ်မှတ်ခြင်းဖြင့် ပြုလုပ်နိုင်ပါသည်။

ထို့ကြောင့် သင်ကြားမှုအတွင်းမှာ ကွန်ယက်သို့ ထည့်မည့် အင်ပွတ်မှာ အချို့အရှည်ရှိသော အက္ခရာများကို encode လုပ်ထားသော အစီအစဉ်ဖြစ်ပြီး၊ output မှာ အတူတူရှည်သော အစီအစဉ်ဖြစ်သော်လည်း တစ်ခုချင်းစီကို တစ်ခုအဆင့်ရွှေ့ထားပြီး `<eos>` ဖြင့် အဆုံးသတ်ထားမည်ဖြစ်သည်။ Minibatch မှာ အစီအစဉ်များစွာ ပါဝင်မည်ဖြစ်ပြီး၊ အားလုံးကို alignment ပြုလုပ်ရန် **padding** ကို အသုံးပြုရမည်ဖြစ်သည်။

အခုတော့ dataset ကို ပြောင်းလဲပေးမည့် function များကို ဖန်တီးကြစို့။ Minibatch အဆင့်မှာ sequence များကို padding ပြုလုပ်လိုသောကြောင့် dataset ကို `.batch()` ခေါ်ပြီး batch လုပ်ပြီးနောက် `map` ကို အသုံးပြု၍ ပြောင်းလဲမှုကို ပြုလုပ်မည်ဖြစ်သည်။ ထို့ကြောင့် ပြောင်းလဲမှု function မှာ minibatch တစ်ခုလုံးကို parameter အဖြစ် လက်ခံမည်:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

ဒီမှာ ကျွန်တော်တို့လုပ်ဆောင်တဲ့ အရေးကြီးအချက်အချို့:

* ပထမဦးဆုံး ကျွန်တော်တို့ string tensor ထဲက အမှန်တကယ် text ကို extract လုပ်ပါတယ်။
* `text_to_sequences` က string တွေကို integer tensor တွေဖြစ်အောင် ပြောင်းပေးပါတယ်။
* `pad_sequences` က အဲ့ဒီ tensor တွေကို အများဆုံးအရှည်အတိုင်း pad လုပ်ပေးပါတယ်။
* နောက်ဆုံးမှာ character တွေကို one-hot encode လုပ်ပြီး shifting နဲ့ `<eos>` ကို ထည့်ပေးပါတယ်။ အဲ့ဒီ character တွေကို one-hot encode လုပ်ရတဲ့ အကြောင်းကို မကြာခင်မှာ ရှင်းပြပေးပါမယ်။

ဒါပေမယ့် ဒီ function က **Pythonic** ဖြစ်ပါတယ်၊ ဒါကြောင့် Tensorflow computational graph အဖြစ် အလိုအလျောက် ပြောင်းလို့မရပါဘူး။ ဒီ function ကို `Dataset.map` function ထဲမှာ တိုက်ရိုက် အသုံးပြုမယ်ဆိုရင် error တွေ ရှိလာနိုင်ပါတယ်။ ဒီ Pythonic call ကို `py_function` wrapper အသုံးပြုပြီး အုပ်ထားဖို့ လိုအပ်ပါတယ်:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Pythonic နှင့် Tensorflow ပြောင်းလဲမှုလုပ်ဆောင်မှုများကို ခွဲခြားဖို့ အလွန်ရှုပ်ထွေးတယ်လို့ ခံစားရနိုင်ပြီး၊ dataset ကို `fit` မှာ ပေးပို့မယ့်အခါမှာ သာမန် Python function တွေကို သုံးပြီး ပြောင်းလဲမှုလုပ်ဆောင်မှုကို မလုပ်တာ ဘာကြောင့်လဲဆိုတာကို မေးမြန်းချင်ဖြစ်နိုင်ပါတယ်။ ဒါကို လုပ်နိုင်တာ သေချာပေမယ့်၊ `Dataset.map` ကို သုံးခြင်းက အလွန်ကြီးမားတဲ့ အကျိုးကျေးဇူးရှိပါတယ်၊ အကြောင်းကတော့ data ပြောင်းလဲမှုလုပ်ဆောင်မှု pipeline ကို Tensorflow computational graph ကို အသုံးပြုပြီး အကောင်အထည်ဖော်တာကြောင့် GPU တွေကို အသုံးချနိုင်ပြီး CPU/GPU အကြား data ပေးပို့မှု လိုအပ်ချက်ကို လျှော့ချနိုင်ပါတယ်။

အခုတော့ generator network ကို တည်ဆောက်ပြီး training စတင်နိုင်ပါပြီ။ ဒါကို အရင် unit မှာ ဆွေးနွေးခဲ့တဲ့ recurrent cell မည်သည့်အမျိုးအစား (simple, LSTM, GRU) ကိုမဆို အခြေခံပြီး တည်ဆောက်နိုင်ပါတယ်။ ဥပမာအနေနဲ့ LSTM ကို အသုံးပြုပါမယ်။

Network က အထိမ်းအမှတ်အဖြစ် character တွေကို input အနေနဲ့ ယူပြီး၊ vocabulary size က အတော်လေးသေးတဲ့အတွက် embedding layer မလိုအပ်ပါဘူး၊ one-hot-encoded input ကို တိုက်ရိုက် LSTM cell ထဲကို ပေးပို့နိုင်ပါတယ်။ Output layer ကတော့ LSTM output ကို one-hot-encoded token number တွေ အဖြစ် ပြောင်းလဲပေးမယ့် `Dense` classifier ဖြစ်ပါတယ်။

ထို့အပြင်၊ variable-length sequences တွေကို ကိုင်တွယ်နေရတဲ့အတွက် `Masking` layer ကို အသုံးပြုပြီး string ရဲ့ padded အပိုင်းကို မထည့်သွင်းဖို့ mask တစ်ခု ဖန်တီးနိုင်ပါတယ်။ ဒါဟာ မဖြစ်မနေလိုအပ်တာ မဟုတ်ပါဘူး၊ အကြောင်းကတော့ `<eos>` token ကို ကျော်လွန်သွားတဲ့ အပိုင်းအားလုံးကို အလွန်စိတ်ဝင်စားနေရတာ မဟုတ်ပေမယ့်၊ layer အမျိုးအစားကို အသုံးပြုတဲ့ အတွေ့အကြုံရဖို့အတွက် သုံးပါမယ်။ `input_shape` က `(None, vocab_size)` ဖြစ်ပြီး၊ `None` က variable length sequence ကို ရည်ညွှန်းပါတယ်၊ output shape ကလည်း `(None, vocab_size)` ဖြစ်ပါတယ်၊ `summary` မှာလည်း မြင်နိုင်ပါတယ်။


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## အထွက်ကို ဖန်တီးခြင်း

အခုတော့ မော်ဒယ်ကို လေ့ကျင့်ပြီးဖြစ်တာကြောင့် အထွက်တစ်ခုခုကို ဖန်တီးဖို့လိုပါတယ်။ ပထမဆုံးအနေနဲ့ token နံပါတ်များစဉ်လိုက်ကိုယ်စားပြုထားတဲ့ စာသားကို ပြန်လည်ဖော်ထုတ်နိုင်တဲ့ နည်းလမ်းတစ်ခုလိုအပ်ပါတယ်။ ဒီအတွက် `tokenizer.sequences_to_texts` function ကို အသုံးပြုနိုင်ပေမယ့် character-level tokenization နဲ့ အလုပ်မလုပ်ကောင်းပါဘူး။ ဒါကြောင့် `word_index` လို့ခေါ်တဲ့ tokenizer ထဲက token တွေရဲ့ dictionary ကိုယူပြီး၊ reverse map တစ်ခုတည်ဆောက်ကာ၊ ကိုယ့်ရဲ့ decoding function ကိုရေးဖို့လိုပါတယ်။


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

အခုတော့ စတင်မည်။ ကျွန်ုပ်တို့မှာ `start` ဆိုတဲ့ string တစ်ခုရှိပြီး၊ အဲဒါကို `inp` sequence အဖြစ် encode လုပ်ပါမည်။ ထို့နောက် တစ်ဆင့်ချင်းစီမှာ ကျွန်ုပ်တို့၏ network ကိုခေါ်ပြီး နောက် character ကို ခန့်မှန်းပါမည်။

Network ရဲ့ output `out` က `vocab_size` elements ရှိတဲ့ vector ဖြစ်ပြီး၊ token တစ်ခုချင်းစီရဲ့ probability ကို ကိုယ်စားပြုထားသည်။ အများဆုံးဖြစ်နိုင်သော token နံပါတ်ကို `argmax` အသုံးပြု၍ ရှာနိုင်ပြီး၊ ဒီ character ကို generated token list ထဲသို့ ထည့်သွင်းပါမည်။ ထို့နောက် generation ကို ဆက်လက်လုပ်ဆောင်ပါမည်။ Character တစ်ခု generate လုပ်တဲ့ ဒီလုပ်ငန်းစဉ်ကို `size` ကြိမ် ထပ်လုပ်ပြီး လိုအပ်သော character အရေအတွက်ကို generate လုပ်ပါမည်။ `eos_token` ကို တွေ့ရှိသောအခါမှာတော့ အစောပိုင်းတွင် generation ကို ရပ်တန့်ပါမည်။


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## လေ့ကျင့်မှုအတွင်း အထွက်ကို နမူနာယူခြင်း

*တိကျမှန်ကန်မှု* ကဲ့သို့သော အသုံးဝင်သော အတိုင်းအတာများမရှိသောကြောင့်၊ မော်ဒယ်က ပိုမိုကောင်းမွန်လာနေကြောင်းကို မြင်နိုင်ရန် တစ်ခုတည်းသော နည်းလမ်းမှာ **နမူနာယူခြင်း** ဖြစ်ပြီး၊ လေ့ကျင့်မှုအတွင်း ထုတ်လုပ်ထားသော စာကြောင်းများကို ကြည့်ရှုရမည်ဖြစ်သည်။ ဒါကို ပြုလုပ်ရန် **callbacks** ကို အသုံးပြုမည်ဖြစ်ပြီး၊ ၎င်းသည် `fit` function သို့ ပေးပို့နိုင်သော function များဖြစ်ပြီး၊ လေ့ကျင့်မှုအတွင်း အချိန်ကာလအလိုက် ခေါ်သုံးမည်ဖြစ်သည်။


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


ဤဥပမာသည် အတော်လေးကောင်းမွန်သောစာသားကို ရAlready ထုတ်ပေးနိုင်ပြီးဖြစ်သော်လည်း အချို့နည်းလမ်းများဖြင့် ထပ်မံတိုးတက်စေနိုင်ပါသည်။

* **ပိုမိုသောစာသား**။ ကျွန်ုပ်တို့၏လုပ်ငန်းအတွက် ခေါင်းစဉ်များကိုသာ အသုံးပြုထားသော်လည်း အပြည့်အစုံသောစာသားများဖြင့် စမ်းသပ်ကြည့်လိုစိတ်ရှိနိုင်ပါသည်။ သတိပြုရန်မှာ RNN များသည် ရှည်လျားသောအကြောင်းအရာများကို ကိုင်တွယ်ရာတွင် အလွန်ကောင်းမွန်သည်မဟုတ်ပါ၊ ထို့ကြောင့် စာကြောင်းတိုများအဖြစ် ခွဲထုတ်ခြင်း သို့မဟုတ် သတ်မှတ်ထားသော `num_chars` (ဥပမာ 256) တန်ဖိုးဖြင့် အမြဲတမ်းတည်ဆောက်ထားသော အတိုင်းအတာအရှည်ဖြင့် လေ့ကျင့်ခြင်းတို့ကို စဉ်းစားသင့်ပါသည်။ အထက်ပါဥပမာကို [အတည်ပြု Keras လေ့ကျင့်ရေးလမ်းညွှန်](https://keras.io/examples/generative/lstm_character_level_text_generation/) ကို အားထား၍ ထိုကဲ့သို့သောဖွဲ့စည်းမှုအဖြစ် ပြောင်းလဲကြည့်နိုင်ပါသည်။

* **Multilayer LSTM**။ LSTM cells 2 သို့မဟုတ် 3 အလွှာကို စမ်းသပ်ကြည့်ရန် make sense ဖြစ်ပါသည်။ ယခင်ယူနစ်တွင် ပြောခဲ့သည့်အတိုင်း LSTM ၏ အလွှာတစ်ခုစီသည် စာသားမှ အချို့သောပုံစံများကို ထုတ်ယူပေးပြီး၊ character-level generator ၏အနေဖြင့် အနိမ့်ဆုံး LSTM အလွှာသည် သရော်အသံများကို ထုတ်ယူရန် တာဝန်ရှိပြီး၊ အမြင့်ဆုံးအလွှာများသည် စကားလုံးများနှင့် စကားလုံးပေါင်းများကို ထုတ်ယူရန် တာဝန်ရှိသည်ဟု မျှော်လင့်နိုင်ပါသည်။ ၎င်းကို LSTM constructor သို့ အလွှာအရေအတွက် parameter ကို ပေးပို့ခြင်းဖြင့် ရိုးရှင်းစွာ အကောင်အထည်ဖော်နိုင်ပါသည်။

* **GRU units** ဖြင့်လည်း စမ်းသပ်ကြည့်လိုစိတ်ရှိနိုင်ပြီး၊ **hidden layer sizes များကို မတူညီစွာ** စမ်းသပ်ကြည့်နိုင်ပါသည်။ Hidden layer အရွယ်အစား အလွန်ကြီးလွန်းပါက overfitting ဖြစ်စေနိုင်ပြီး (ဥပမာ network သည် စာသားကို တိတိကျကျ သင်ယူသွားမည်ဖြစ်သည်)၊ အရွယ်အစား သေးလွန်းပါက ကောင်းမွန်သောရလဒ် မထုတ်ပေးနိုင်ပါ။


## Soft text generation and temperature

ယခင် `generate` ကိုဖော်ပြထားသော အဓိပ္ပါယ်အရ၊ ကျွန်ုပ်တို့သည် အမြဲတမ်း အမြင့်ဆုံး probability ရှိသော အက္ခရာကို ရွေးပြီး ထုတ်လုပ်ထားသော စာသားတွင် နောက်ထပ် အက္ခရာအဖြစ် အသုံးပြုခဲ့သည်။ ဒီနည်းလမ်းကြောင့် စာသားသည် အချို့သော အက္ခရာအလှည့်အပြောင်းများကို ထပ်တလဲလဲ ဖြစ်စေခဲ့သည်။ ဥပမာအားဖြင့်:
```
today of the second the company and a second the company ...
```

သို့သော်၊ နောက်ထပ် အက္ခရာအတွက် probability distribution ကိုကြည့်မယ်ဆိုရင်၊ အမြင့်ဆုံး probability ရှိသော အက္ခရာများအကြား ကွာဟချက်သည် များစွာ မကြီးမားနိုင်ပါ။ ဥပမာအားဖြင့် အက္ခရာတစ်ခုမှာ probability 0.2 ရှိနိုင်ပြီး၊ နောက်တစ်ခုမှာ 0.19 ရှိနိုင်သည်။ ဥပမာအားဖြင့် '*play*' ဆိုသော စာကြောင်းတွင် နောက်ထပ် အက္ခရာသည် space ဖြစ်နိုင်သလို၊ **e** (ဥပမာ player ဆိုသော စကားလုံးတွင်) ဖြစ်နိုင်ပါသည်။

ဒီအခြေအနေကြောင့် အမြင့်ဆုံး probability ရှိသော အက္ခရာကိုသာ ရွေးချယ်ခြင်းသည် အမြဲတမ်း "တရားမျှတ" မဖြစ်နိုင်ပါ။ ဒုတိယမြင့်ဆုံး probability ရှိသော အက္ခရာကို ရွေးချယ်ခြင်းသည်လည်း အဓိပ္ပါယ်ရှိသော စာသားကို ရရှိစေနိုင်ပါသည်။ ထို့ကြောင့် network output မှပေးသော probability distribution ကို အသုံးပြု၍ **sample** လုပ်ခြင်းသည် ပိုပြီး ဉာဏ်ရှိသော နည်းလမ်းဖြစ်သည်။

ဒီ sampling ကို **multinomial distribution** ဟုခေါ်သော နည်းလမ်းကို အကောင်အထည်ဖော်ပေးသော `np.multinomial` function ကို အသုံးပြု၍ ပြုလုပ်နိုင်ပါသည်။ **soft** text generation ကို အကောင်အထည်ဖော်ပေးသော function ကို အောက်တွင် ဖော်ပြထားသည်:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

ကျွန်ုပ်တို့သည် **အပူချိန်** ဟုခေါ်သော နောက်ထပ်ပါရာမီတာတစ်ခုကို မိတ်ဆက်ပေးခဲ့ပြီး၊ ၎င်းကို အမြင့်ဆုံးဖြစ်နိုင်မှုကို မည်မျှတင်းကြပ်စွာလိုက်နာရမည်ကို ဖော်ပြရန် အသုံးပြုသည်။ အပူချိန်သည် 1.0 ဖြစ်ပါက၊ ကျွန်ုပ်တို့သည် တရားမျှတသော multinomial sampling ကို ပြုလုပ်ပြီး၊ အပူချိန်သည် အဆုံးမရှိအထိ မြင့်တက်သွားသောအခါ - ဖြစ်နိုင်မှုအားလုံးသည် တူညီသွားပြီး၊ နောက်တစ်ခုသော အက္ခရာကို ကျပန်းရွေးချယ်သည်။ အောက်တွင်ပေးထားသော ဥပမာတွင် ကျွန်ုပ်တို့သည် အပူချိန်ကို အလွန်များစွာမြှင့်တင်သောအခါ၊ စာသားသည် အဓိပ္ပါယ်မရှိသွားကြောင်းကို တွေ့နိုင်ပြီး၊ အပူချိန်သည် 0 အနီးသို့ ရောက်လာသောအခါ "cycled" အခက်အခဲဖြင့် ဖန်တီးထားသော စာသားနှင့် ဆင်တူသည်ကို တွေ့နိုင်သည်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါရှိနိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှု ဝန်ဆောင်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွဲအချော်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
